In [20]:
import pandas as pd
from datetime import datetime
import math

In [21]:
df = pd.read_csv("../00_data/merged_all_by_algo.csv", sep=";", decimal=",")
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.set_index('datetime')
df.head(3)

,date,hour,day_ahead_price,intraday_price,price_diff,solar_capacity,solar_generation_actual,wind_capacity,wind_generation_actual,wind_generation_forecast,solar_generation_forecast
datetime,,,,,,,,,,,
2016-01-01 00:00:00,2016-01-01,0,23.86,29.34,5.48,39021,0,44325,8648,6033,0
2016-01-01 01:00:00,2016-01-01,1,22.39,24.28,1.89,39021,0,44325,8579,7421,0
2016-01-01 02:00:00,2016-01-01,2,20.59,25.26,4.67,39021,0,44325,8542,7266,0


In [22]:
def determineSeason(inputMonth):
    if inputMonth == 12 or inputMonth <= 2: # winter
        return 0
    elif inputMonth > 2 and inputMonth <= 5: # spring
        return 1
    elif inputMonth > 5 and inputMonth <= 8: # summer
        return 2
    elif inputMonth > 8 and inputMonth <= 11: # autumn
        return 3
    return -1 # error

df["total_generation_forecast"] = df["wind_generation_forecast"] + df["solar_generation_forecast"]
df["total_generation_actual"] = df["solar_generation_actual"]+ df["wind_generation_actual"]
df["wind_forecast_error"] = df["wind_generation_actual"] - df["wind_generation_forecast"]
df["solar_forecast_error"] = df["solar_generation_actual"] - df["solar_generation_forecast"]
df["forecast_error"] = df["total_generation_actual"] - df["total_generation_forecast"]

print(df.head(5))
# all moving averages are calculated with exponential smoothing
# moving average 30 days of forecast error
df["mov_avg_30_forecast_error"] = df.forecast_error.ewm(span=30*24,  min_periods=1).mean()
df["mov_avg_30_wind_forecast_error"] = df.wind_forecast_error.ewm(span=30*24,  min_periods=1).mean()
df["mov_avg_30_solar_forecast_error"] = df.solar_forecast_error.ewm(span=30*24,  min_periods=1).mean()
# mov avg 10 days
df["mov_avg_10_forecast_error"] = df.forecast_error.ewm(span=10*24,  min_periods=1).mean()
df["mov_avg_10_wind_forecast_error"] = df.wind_forecast_error.ewm(span=10*24,  min_periods=1).mean()
df["mov_avg_10_solar_forecast_error"] = df.solar_forecast_error.ewm(span=10*24,  min_periods=1).mean()
# forecast error avg 24 hours
df["mov_avg_1_forecast_error"] = df.forecast_error.ewm(span=24,  min_periods=1).mean()
df["mov_avg_1_wind_forecast_error"] = df.wind_forecast_error.ewm(span=24,  min_periods=1).mean()
df["mov_avg_1_solar_forecast_error"] = df.solar_forecast_error.ewm(span=24,  min_periods=1).mean()
# price avg 24 hours
df["mov_avg_1_day_ahead"] = df.day_ahead_price.ewm(span=24,  min_periods=1).mean()
df["mov_avg_1_intraday"] = df.intraday_price.ewm(span=24,  min_periods=1).mean()

# Add column with 24h max
df["solar_24h_max"] = df.solar_generation_actual.rolling(24,  min_periods=1).max()
df["wind_24h_max"] = df.wind_generation_actual.rolling(24,  min_periods=1).max()

# Calculate a moving average solar FOR EACH HOUR: MOV_AVG_Thur <-- AVG(Mon 1am, Tue 1am, Wed 1am, Thur 1am)
for i in range (24):
    df.loc[df.index.hour == i,"hourly_moving_avg_solar"] = df.loc[df.index.hour == i,"solar_generation_actual"].rolling(4,  min_periods=1).mean()
    df.loc[df.index.hour == i,"hourly_moving_avg_2_solar"] = df.loc[df.index.hour == i,"solar_generation_actual"].rolling(2,  min_periods=1).mean()
    df.loc[df.index.hour == i,"hourly_moving_avg_7_solar"] = df.loc[df.index.hour == i,"solar_generation_actual"].rolling(7,  min_periods=1).mean()
    
# Calculate a moving average wind FOR EACH HOUR: MOV_AVG_Thur <-- AVG(Mon 1am, Tue 1am, Wed 1am, Thur 1am)
for i in range (24):
    df.loc[df.index.hour == i,"hourly_moving_avg_wind"] = df.loc[df.index.hour == i,"wind_generation_actual"].rolling(4,  min_periods=1).mean()
    df.loc[df.index.hour == i,"hourly_moving_avg_2_wind"] = df.loc[df.index.hour == i,"wind_generation_actual"].rolling(2,  min_periods=1).mean()
    df.loc[df.index.hour == i,"hourly_moving_avg_7_wind"] = df.loc[df.index.hour == i,"wind_generation_actual"].rolling(7,  min_periods=1).mean()
    

# season and weekday calculation
df["season"] = df.index.month.map(lambda x: determineSeason(x))
df["month"] = df.index.month
df["weekday"] = df.index.weekday
df.drop(columns=["date"], inplace=True, axis=1)
print(df.head(5))

df.to_csv("../00_data/df_with_calc_fields.csv", sep=";", decimal=",")
print("done")

                           date  hour  day_ahead_price  intraday_price  \
datetime                                                                 
2016-01-01 00:00:00  2016-01-01     0            23.86           29.34   
2016-01-01 01:00:00  2016-01-01     1            22.39           24.28   
2016-01-01 02:00:00  2016-01-01     2            20.59           25.26   
2016-01-01 03:00:00  2016-01-01     3            16.81           20.24   
2016-01-01 04:00:00  2016-01-01     4            17.41           21.94   

                     price_diff  solar_capacity  solar_generation_actual  \
datetime                                                                   
2016-01-01 00:00:00        5.48           39021                        0   
2016-01-01 01:00:00        1.89           39021                        0   
2016-01-01 02:00:00        4.67           39021                        0   
2016-01-01 03:00:00        3.43           39021                        0   
2016-01-01 04:00:00      